# Train Two-layer NN with MNIST

In [3]:
from torchvision import datasets
from tqdm import tqdm
import numpy as np

In [4]:
def load_mnist(normalize = True, flatten = True, one_hot_label = True):
  mnist_train = datasets.MNIST(root = "./data/", train=True, download=True)
  mnist_test = datasets.MNIST(root = './data/', train=False, download=True)
  print ("mnist_train:\n",mnist_train,"\n")
  print ("mnist_test:\n",mnist_test,"\n")
  print ("Done.")

  mnist_train_img = []
  mnist_train_label = []
  mnist_test_img = []
  mnist_test_label = []

  for train_data in mnist_train:
    train_img = np.array(train_data[0]).astype(np.uint8)
    train_img = train_img.reshape(784)
    mnist_train_img.append(train_img)
    mnist_train_label.append(train_data[1])

  for test_data in mnist_test:
    test_img = np.array(test_data[0]).astype(np.uint8)
    test_img = test_img.reshape(784)
    mnist_test_img.append(test_img)
    mnist_test_label.append(test_data[1])

  mnist_train_img, mnist_train_label = np.array(mnist_train_img), np.array(mnist_train_label)
  mnist_test_img, mnist_test_label = np.array(mnist_test_img), np.array(mnist_test_label)

  if normalize:
    mnist_train_img, mnist_test_img = mnist_train_img.astype(np.float32), mnist_test_img.astype(np.float32)
    mnist_train_img /= 255.0
    mnist_test_img /= 255.0

  if one_hot_label:
    new_train_label = []
    new_test_label = []
    for train_label in mnist_train_label:
      fix_label = np.zeros(10)
      fix_label[train_label] = 1
      new_train_label.append(fix_label)
    
    for test_label in mnist_test_label:
      fix_label = np.zeros(10)
      fix_label[test_label] = 1
      new_test_label.append(fix_label)

    mnist_train_label, mnist_test_label = np.array(new_train_label).astype(np.uint8), np.array(new_test_label).astype(np.uint8)
  
  print("load MNIST done")
  return (mnist_train_img, mnist_train_label), (mnist_test_img, mnist_test_label)
    

In [5]:
def sigmoid(x):
  return 1 / (1+np.exp(-x))

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

def numerical_gradient(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)  # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x)  # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val  # 값 복원
        it.iternext()

    return grad

In [6]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init=0.01):
    self.parameter = {}
    self.parameter["W1"] = weight_init *  np.random.rand(input_size, hidden_size)
    self.parameter["b1"] = np.zeros(hidden_size)
    self.parameter["W2"] = weight_init * np.random.rand(hidden_size, output_size)
    self.parameter["b2"] = np.zeros(output_size)

  def predict(self,x):
    W1, W2 = self.parameter["W1"], self.parameter["W2"]
    b1, b2 = self.parameter['b1'], self.parameter['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)
    return y

  def loss(self, x, t):
    predict = self.predict(x)
    return cross_entropy_error(predict,t)

  def accuracy(self, x, t):
    predict = self.predict(x)
    predict = np.argmax(predict, axis=1)
    actual = np.argmax(t, axis=1)

    accuracy = np.sum(predict == actual) / float(x.shape[0])
    return accuracy

  def numerical_gradient(self,x,t):
    loss_W = lambda W: self.loss(x,t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.parameter["W1"])
    grads['b1'] = numerical_gradient(loss_W, self.parameter['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.parameter['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.parameter['b2'])
    return grads

In [9]:
# 학습구현

def train(batch_size, iter_num, lr=0.1):
  network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)
  train_loss = []
  train_acc_list = []
  test_acc_list = []
  (train_img, train_label), (test_img, test_label) = load_mnist()
  iter_epoch = max(train_img.size/batch_size, 1)

  for i in tqdm(range(iter_num)):
    batch = np.random.choice(train_img.shape[0], batch_size)
    batch_img = train_img[batch]
    batch_label = train_label[batch]

    grad = network.numerical_gradient(batch_img, batch_label)
    for key in ('W1', 'b1', 'W2', 'b2'):
      network.parameter[key] -= lr * grad[key]

    loss = network.loss(batch_img, batch_label)
    train_loss.append(loss)

    train_acc = network.accuracy(train_img, train_label)
    test_acc = network.accuracy(test_img, test_label)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train_acc, test_acc : " + str(train_acc) + " , "  + str(test_acc))

In [10]:
train(100, 100)

mnist_train:
 Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train 

mnist_test:
 Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data/
    Split: Test 

Done.
load MNIST done


  1%|          | 1/100 [01:16<2:06:39, 76.76s/it]

train_acc, test_acc : 0.09863333333333334 , 0.0958


  2%|▏         | 2/100 [02:32<2:04:50, 76.44s/it]

train_acc, test_acc : 0.0993 , 0.1032


...

train_acc, test_acc : 0.10218333333333333 , 0.101


 12%|█▏        | 12/100 [15:15<1:51:52, 76.28s/it]

train_acc, test_acc : 0.10218333333333333 , 0.101


 12%|█▏        | 12/100 [15:19<1:52:21, 76.61s/it]


KeyboardInterrupt: ignored